In [ ]:
# First, in order to be able to reach any floor from any other floor, two conditions must be met:

# 1: The sum of the upward interval and the downward interval is less than or equal to the total number of floors
# If this condition were not met, there would be some floor(s) on which you can move neither up nor down

# 2: The greatest common factor between the upward interval and downward interval is 1
# If this condition were not met, some floor(s) could not be reached starting from floor 1 (and vice versa)

# This function below checks if all floors can be reached starting from floor one
# It takes 3 positional arguments and an optional "verbose" keyword argument

def all_floors_can_be_reached(Up_Move_Num_Floors, Down_Move_Num_Floors, Total_Building_Floors, verbose=True):
    
    # --------------------------------------------------------------------------------------------------
    
    # First make sure there are enough floors in the building
    # If not, the function returns False
    
    if((Up_Move_Num_Floors + Down_Move_Num_Floors) > Total_Building_Floors):
        
        if(verbose == True):
            print("Not all floors can be reached")
            
        return False
    
    # --------------------------------------------------------------------------------------------------
    
    # If there are enough floors, the function finds the greatest common factor
    else:
        
    # --------------------------------------------------------------------------------------------------
        
        # This method of calculating the greatest common factor is not the most intuitively obvious,
        # but it works. In a future commit, I may include a description of why it works as well as
        # uploade code for more intuitive methods of calculating it.
        
        Remainder_From_Up = Up_Move_Num_Floors
        Remainder_From_Down = Down_Move_Num_Floors
        
        while(Remainder_From_Up != Remainder_From_Down):
            
            if(Remainder_From_Up > Remainder_From_Down):
                Remainder_From_Up -= Remainder_From_Down
            else:
                Remainder_From_Down -= Remainder_From_Up
                
        Greatest_Common_Factor = Remainder_From_Up
        
    # --------------------------------------------------------------------------------------------------
                
        # If the greatest common factor is 1, the function returns True, otherwise it returns False
        
        if(Greatest_Common_Factor == 1):
            
            if(verbose == True):
                print("All floors can be reached")
            
            return True
        
        else:
            
            if(verbose == True):
                print("Not all floors can be reached")
            
            return False

In [ ]:
# With this we can verify that we can reach all floors in the example:
all_floors_can_be_reached(7, 9, 16)

In [ ]:
def number_of_moves(Destination_Floor, Up_Interval, Down_Interval):
    
    # This function returns the total moves to reach a floor starting from the ground (Floor 1)
    # The function assumes all floors are reachable
    
    # --------------------------------------------------------------------------------------------------
    
    Total_Moves = 0
    Floors_To_Ascend = Destination_Floor - 1
    
    # In this function, a "baseline" floor is any floor separated by a factor of the Up_Interval from the
    # ground floor. This means it can be reached with only "up" moves, and thus with fewer moves than any
    # floor above it.
    
    # This is useful because the moves to a baseline floor can be easily found, and total moves can be 
    # found by adding the moves to the baseline plus moves from baseline to destination.
    
    # For exmaple, say your destination floor is floor 13 (Up_Interval=7, Down_Interval=9). You can get
    # to floor 8 in 1 move up. Floor 13 is 5 floors above the baseline 8, so the number of moves to floor
    # 13 is one higher than the number of moves to 6 (6 is 5 higher than 1). The number of moves to floor 
    # 20 (20 is 5 higher than baseline 15) is 2 more than the number of moves to 6. And so on.
    
    # --------------------------------------------------------------------------------------------------
    
    Moves_To_Baseline = (Floors_To_Ascend//Up_Interval)
    Total_Moves += Moves_To_Baseline
    
    Remaining_Floors_Up = Floors_To_Ascend - (Moves_To_Baseline * Up_Interval)
    
    # --------------------------------------------------------------------------------------------------
    
    # If you are at a baseline, getting to a floor between your current floor and the next baseline up
    # will require both up and down moves. The function populates the Moves_To_Remainder dictionary with
    # values indicating how many extra moves it takes to go more N floors up from the baseline (where N
    # is the key). It starts out populated with 0:0 because if your destination floor is the baseline 
    # you are on, you need to go up 0 additional floors and need 0 more moves.
    
    Moves_To_Remainder = {0:0}
    
    Curr_Above_Baseline = 0
    Extra_Moves = 0
    
    while(len(Moves_To_Remainder) < Up_Interval):
        
        while(Curr_Above_Baseline <= Down_Interval):
            Curr_Above_Baseline += Up_Interval
            Extra_Moves += 1
            
        Curr_Above_Baseline -= Down_Interval
        Extra_Moves += 1
        
        Moves_To_Remainder[Curr_Above_Baseline] = Extra_Moves
        
    # --------------------------------------------------------------------------------------------------
        
    # Once this while loop is complete, the dictionary will have a value for every possible number of floors
    # above the baseline. Since the function already calculated the number of up-moves to get to the baseline,
    # we can get the total moves by adding the moves required to get from the baseline to the destination
    
    Total_Moves += Moves_To_Remainder[Remaining_Floors_Up]

    return Total_Moves

In [ ]:
# Since there are only 100 floors, you can test this function for each floor and see if it matches expectations
# as well as see which floor it is saying requires the most moves

Test_Destination_Floor = 1

for Test_Destination_Floor in range(1, 101):
    Moves_Needed = number_of_moves(Test_Destination_Floor, 7, 9)
    print("Floor", Test_Destination_Floor, "needs", Moves_Needed, "moves to reach")

In [ ]:
# In general, it may not be efficient to check every floor, and in some cases, some floors may not be reachable
# The floor_with_max_moves() function finds the floor that requires the most moves in the general case

def floor_with_max_moves(Up_Interval, Down_Interval, Total_Floors, verbose=True):
    
    # --------------------------------------------------------------------------------------------------
    
    # First this function calls the all_floors_can_be_reached() function to verify that all floors are
    # reachable. If they are not, the function indicates this and returns None.
    
    if(all_floors_can_be_reached(Up_Interval, Down_Interval, Total_Floors, verbose=False) == True):
        
        # ----------------------------------------------------------------------------------------------
        
        # For tall buildings, this function will only look at a set of higher floors, starting from the 
        # highest baseline floor that is guaranteed to still has a full up-interval of floors above it.
        
        Full_Baseline_Intervals = (Total_Floors - 1)//Up_Interval
        Start_Baseline = (Up_Interval * (Full_Baseline_Intervals - 1)) + 1
        
        if(verbose == True):
            print("First floor checked:", Start_Baseline)
            
        # ----------------------------------------------------------------------------------------------
        
        # The function then checks the number of moves for each floor within the already-determined set of
        # upper floors until it reaches the top. It keeps track of which floor took the maximum number of
        # moves and how many it took. If another floor takes more than that, it becomes the new maximum.
        
        # It calls the number_of_moves() function to calculate this.
        
        Current_Floor_Check = Start_Baseline
        Floor_With_Max_Moves = Start_Baseline
        Current_Max = 0 
        
        while(Current_Floor_Check <= Total_Floors):
            
            Moves_To_Floor_Being_Checked = number_of_moves(Current_Floor_Check, Up_Interval, Down_Interval)
            
            if(Moves_To_Floor_Being_Checked > Current_Max):
                Current_Max = Moves_To_Floor_Being_Checked
                Floor_With_Max_Moves = Current_Floor_Check
            
            Current_Floor_Check += 1
            
        # ----------------------------------------------------------------------------------------------
        
        # The function returns the floor with the maximum number of moves, and the number required, as a list
            
        if(verbose == True):
            print("Floor", Floor_With_Max_Moves, "in", Current_Max, "moves")
            
        return [Floor_With_Max_Moves, Current_Max]
        
    else:
        if(verbose == True):
            print("Not all floors can be reached")
            
        return None

In [ ]:
# With this we can get an answer to the example of 100 floors with an up-interval of 7 and down-interval of 9
floor_with_max_moves(7, 9, 100)

In [ ]:
# The previous function works and illustrates the logical process for answering these types of questions
# However, it is not the most efficient way to do this since it involves duplication of some calculations
# The same Moves_To_Remainder dictionary gets generated each time the number_of_moves() function is called

# Here is a function that uses the same idea but more efficiently

# It only generates the Moves_To_Remainder dictionary once, and takes advantage of the fact that the floor
# requiring the maximum number of moves will always be above a baseline floor by a number of floors that 
# matches the last entry in that dictionary

def floor_with_max_moves_faster(Up_Interval, Down_Interval, Total_Floors, verbose=True):
    
    if(all_floors_can_be_reached(Up_Interval, Down_Interval, Total_Floors, verbose=False) == True):
        
    # --------------------------------------------------------------------------------------------------    
    
    # This generates the same dictionary as the number_of_moves() function, but only once instead of 
    # calling that function
        
        Moves_To_Remainder = {0:0}
    
        Curr_Above_Baseline = 0
        Extra_Moves = 0

        while(len(Moves_To_Remainder) < Up_Interval):

            while(Curr_Above_Baseline <= Down_Interval):
                Curr_Above_Baseline += Up_Interval
                Extra_Moves += 1

            Curr_Above_Baseline -= Down_Interval
            Extra_Moves += 1

            Moves_To_Remainder[Curr_Above_Baseline] = Extra_Moves
            
    # --------------------------------------------------------------------------------------------------
    
    # The function finds how many baseline floors you can go up while still having a final destination floor 
    # that is still the right number of floors above that baseline.
            
        Full_Baseline_Intervals = (Total_Floors - (Curr_Above_Baseline + 1))//Up_Interval
        Max_Moves = Full_Baseline_Intervals + Extra_Moves
        Floor_With_Max_Moves = (Full_Baseline_Intervals * Up_Interval) + (Curr_Above_Baseline + 1)
        
        if(verbose == True):
            print("Floor", Floor_With_Max_Moves, "in", Max_Moves, "moves")
            
        return [Floor_With_Max_Moves, Max_Moves]
        
    else:
        
        if(verbose == True):
            print("Not all floors can be reached")
            
        return None

In [ ]:
# This gets the same answer for the example problem
floor_with_max_moves_faster(7, 9, 100)